**REMARQUE : la date limite pour envoyer votre travail à mghesmoune@gmail.com est le vendredi 28 janvier.**
## Que signifie mettre un modèle en production ?

D'une manière générale et sans entrer dans beaucoup de détails, la mise en production d'un modèle est un processus dans lequel un modèle est intégré dans un environnement informatique existant et mis à disposition d'autres équipes pour utilisation et consommation.
Pour rendre ce processus aussi efficace que possible, de nombreuses étapes doivent être soigneusement gérées : stockage des données, service, journalisation des artefacts, gestion des ressources, pour n'en nommer que quelques-unes.

Ce cours est destiné à vous aider à démarrer la mise en production de vos modèles avec des API.

## Qu'est-ce qu'une API exactement ?

C'est l'abréviation d'Application Programming Interface. C'est simplement un intermédiaire entre deux applications indépendantes qui communiquent entre elles.
Si vous êtes un développeur et que vous souhaitez rendre votre application disponible pour que d'autres développeurs puissent l'utiliser et l'intégrer, vous créez une API qui sert de point d'entrée à votre application.
Les développeurs devront donc communiquer avec cette API via des requêtes HTTP pour consommer et interagir avec votre service.

Considérez les API comme une abstraction de votre application (les utilisateurs ne voient pas votre code et n'installent rien, ils appellent simplement l'API) et une simplification de l'intégration avec des tiers (développeurs, applications, autres API, etc.)
Les API sont utilisées partout. Le monde de la science des données ne fait pas exception.

## Qu'est-ce que FastAPI et pourquoi est-il adapté à l'apprentissage automatique en production ?

FastAPI est actuellement le framework incontournable pour créer des API robustes et hautes performances qui évoluent dans les environnements de production.
https://fastapi.tiangolo.com/

FastAPI a gagné en popularité ces derniers temps et a vu une énorme augmentation de l'adoption par les utilisateurs parmi les développeurs Web, mais aussi les data scientists et les ingénieurs ML.
La **syntaxe de FastAPI est simple**, ce qui le rend rapide à utiliser.
Servir l'API est la responsabilité d'**uvicorn** (https://www.uvicorn.org/), ce qui est un bon choix étant donné qu'uvicorn est une implémentation de serveur ASGI ultra-rapide, utilisant *uvloop* et *httptools*.

## Premiers pas avec FastAPI

Dépendances logicielles requises pour FastAPI :

     Python 3.6+
     FastAPI
     Serveur ASGI pour la production comme Uvicorn ou Hypercorn.
     Pydantic
     
Installer les dépendances - dans un Terminal, taper les trois commandes suivantes :

   `pip install fastapi`;  
   `pip install uvicorn`;
   `pip install pydantic`

### Basic Hello World
Tout d'abord, nous allons faire une implémentation très basique pour nous familiariser avec FastAPI.
```
from fastapi import FastAPI
app = FastAPI()
@app.get("/")
def index():
    return {"message": "Hello World"}
```

Enregistrez ce programme dans main.py puis depuis un terminal, exécutez :
`uvicorn main:app --reload`

Le message suivant doit être imprimé :
```
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:     Started reloader process [28720]
INFO:     Started server process [28722]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
```
Dans un navigateur web, tapez l'url `http://127.0.0.1:8000`. Vous devez voir `{"message":"Hello World"}`

### Documentation interactive
`http://127.0.0.1.8000/docs` affichera la documentation interactive de Swagger UI.

### Aller plus loin
Jetez un oeil sur la documentation de FastAPI.

## FastAPI pour les modèles de ML

Téléchargez le **dataset** d'ici : https://github.com/Quicksign/ocrized-text-dataset/releases
Travaillons dans un premier temps avec le dataset *QS-OCR-small.tar.gz*. Décompressez l'archive.
Ce dataset contient plusieurs types (classes) de documents : ADVE, Email, Form, Letter, etc.
Il faut savoir que les documents originaux sont des images ou PDFs scannés contenant du texte.
On extrait le texte d'une image/PDF scanné à l'aide d'un moteur OCR (https://fr.wikipedia.org/wiki/Reconnaissance_optique_de_caract%C3%A8res), par exemple Tesseract (https://github.com/tesseract-ocr/tesseract).

### Entrainez un classifier de texte avec SVM
Dans le notebook :

1. écrire une fonction python permettant de lire un fichier texte : elle prendra en entrée le chemin du fichier texte puis elle retournera son contenu.
```
def get_texte(filepath):
      ____
      return content
```

2. écrire une fonction python permettant de retourner le type (classe) de document (parmi ADVE, Email, etc.) depuis son chemin.
```
def get_type_doc(filepath):
      ____
      return type_doc
```
par exemple : `get_type_doc("../Data/data6/QS-OCR-small/News/50288669-8669.txt")` doit retourner `"News"`

3. sauvegarder les textes et leurs types dans les variables X_text, y_text
```
from glob import glob
X_text = []
y_text = []
path_data = "../Data/data6/QS-OCR-small"
for fichier_txt in glob(path_data+"/*/*"):
       X_text...
       y_text...
```

4. Vectorisation de texte avec tf-idf :
La vectorisation de texte est le processus de conversion de texte en représentation numérique.
Pour ce faire, nous allons utiliser l'implémentation scikit-learn de TF-IDF : https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
En bref, la vectorisation tf-idf donne une représentation numérique des mots entièrement dépendante de la nature et du nombre de documents considérés. Les mêmes mots auront des représentations vectorielles différentes dans un autre corpus.
```
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(...)
x_train_vec = vectorizer.fit_transform(X_train).toarray()
x_test_vec = vectorizer.transform(X_test).toarray()
#pensez à normaliser les données si nécessaire
```

5. utiliser la validation croisée (https://scikit-learn.org/stable/modules/cross_validation.html) pour choisir les meilleures valeurs des hyper-paramètres du classifier
```
from sklearn.model_selection import StratifiedKFold
vectorizer = TfidfVectorizer(...)
#initialiser un classifier SVM
skf = StratifiedKFold(n_splits=...)
for train_index, test_index in skf.split(X, y):
       X_train, X_test = X[train_index], X[test_index]
       y_train, y_test = y[train_index], y[test_index]
       #vectorisation
       #normaliser les données si nécessaire
       #entrainer le classifier sur x_train_vec
       #faire les prédictions sur le jeu de test
       #évaluer votre classifier en calculant des métriques tel que : accuracy, precision, recall
#calculer et afficher la moyenne et l'écart type sur chacune des métriques caculée dans la boucle for précédente
```

6. répéter l'étape 5. plusieurs fois en modifiant les paramètres du TfidfVectorizer et du SVM

7. en reprenant les valeurs des paramètres du TfidfVectorizer et du SVM qui ont donné les plus grandes valeurs des métriques, entrainer un nouveau classifier SVM sur **tout** le jeu de données.

8. sauvegarder le vectorizer et le SVM dans des fichiers pickle.

   pour sauvegarder le vectorizer:
```
import pickle as pkl
with open("vectorizer.pkl", "wb") as f:
        pkl.dump(vectorizer, f)
```

### Créer un Web Service avec FastAPI
L'idée est de créer un web service permettant de classifier un fichier texte passé en paramètre.
Les principales étapes sont les suivantes:
1. charger les fichiers pickle du vectorize et du SVM.
2. définir une méthode POST qui prend un fichier texte et retourne sa classe :
```
@app.post("/Classifier")
def Classification(
        txt_file: UploadFile = File(...),
    ):
        """
        Description: POST request to classify the document
        ### Input POST
        - **txt_file** (buffer): text document
        ### Output
        - **class_doc** (string): string response of POST request containing the predicted class of the file
        """
        - lire le contenu du fichier texte
        - appliquer la vectorisation 
        `text_vec = vectorizer.transform([text]).toarray()`
        - normaliser si nécessaire
        - predire la classe du fichier texte
        ...
```

### Tester votre classifier via votre web-service
En utilisant le Swagger UI, tester votre classifier sur plusieurs fichiers texte (vous pouvez en choisir quelques-uns à partir du dossier QS-OCR-Large.tar.gz https://github.com/Quicksign/ocrized-text-dataset/releases)

### déployer votre web-service sur un serveur cloud
pour que votre web service soit accéssible à tout le monde (vos clients), il faut le déployer sur un serveur, cloud par exemple, puis partager l'URL de votre web service à vos clients pour qu'ils puissent l'utiliser afin de classifier automatiquement leurs documents.